# Parcial 3 ANALNUM

Códigos del parcial, correr en octave porque aquí no funcionan tan bien, entregado por:

    * Sergio Bello Torres
    * Santiago Ochoa Quiroga
    * Mateo Manosalva Amaris.

## Punto 2

In [ ]:
A=[2, 0, -4, 1];
raices=Newton_Raphson(A,10^-6);
disp('raíces del polinomio 2x^3-4x+1')
disp(raices);

u=zeros(length(raices),1);
for i=1:length(raices)
    u(i)=pol(A,raices(i));
end

disp('valor del polinomio en cada raíz')
disp(u)

function p=Newton_Raphson(a,tol)
    n=length(a);
    p=zeros(1,n-1);
    for i=1:n-2
    u=sum(abs(a(2:end))./abs(a(1)),'all');
    M=max([1,u]);
    error=1;
    r=M;
        while error >  tol
        r_1=r;
        r=r-pol(a,r)/der_pol(a,r);
        error=abs(r_1-r);
        end
    p(i)=r;
    a=Ruffini(a,r);
    end
    p(n-1)=-a(2)/a(1);
end

function f = pol(a,x)
    n=length(a);
    f=a(1);
    for i=2:n
        f=x*f+a(i);
    end
end

function f= der_pol(a,x)
    n=length(a);
    f=(n-1)*a(1);
    for i=2:n-1
        f=x*f+(n-i)*a(i);
    end
end

function f = Ruffini(a,x)
    n=length(a);
    f=zeros(1, n-1);
    u=a(1);
    f(1)=a(1);
    for i=2:n-1
        u=x*u+a(i);
        f(i)=u;
    end
end

## Punto 4

In [ ]:
figure('position', get(0, 'screensize'))
axes('position', [0 0 1 1])
[x, y] = ginput();
plot(x, y, 'bo-', 'LineWidth', 1.5, 'MarkerSize', 8);
hold on;
t = 1:length(x);
t_fine = linspace(1, length(x), 1000);
x_spline = spline(t, x, t_fine);
y_spline = spline(t, y, t_fine);
plot(x_spline, y_spline, 'g--');
dx = zeros(length(x)-2, 1);
dy = zeros(length(x)-2, 1);
for i = 1:length(x)-2
    dx(i) = x(i+2) - 2*x(i+1) + x(i);
    dy(i) = y(i+2) - 2*y(i+1) + y(i);
end
dx = 6*dx;
dy = 6*dy;
% Matriz de splines para algoritmo de Thomas
n = length(dx);
c = ones(n-1, 1);  % Subdiagonal
d = 4 * ones(n, 1); % Diagonal
a = ones(n-1, 1);  % Superdiagonal
% Eliminación hacia adelante
for i = 2:n
    m = c(i-1) / d(i-1);
    d(i) = d(i) - m * a(i-1);
    dx(i) = dx(i) - m * dx(i-1);
    dy(i) = dy(i) - m * dy(i-1);
end
% Sustitución hacia atrás para sigma_x
sigma_x = zeros(n, 1);
sigma_x(n) = dx(n) / d(n);
for i = n-1:-1:1
    sigma_x(i) = (dx(i) - a(i) * sigma_x(i+1)) / d(i);
end
% Sustitución hacia atrás para sigma_y
sigma_y = zeros(n, 1);
sigma_y(n) = dy(n) / d(n);
for i = n-1:-1:1
    sigma_y(i) = (dy(i) - a(i) * sigma_y(i+1)) / d(i);
end
sigma_x = [0; sigma_x; 0];
sigma_y = [0; sigma_y; 0];
x_int = [];
y_int = [];
for i = 1:length(x)-1
    t_i = linspace(t(i), t(i+1), 1000);
    for k = 1:length(t_i)
        x_val = spline_polinomios(t_i(k), t(i), t(i+1), x(i), x(i+1), sigma_x(i), sigma_x(i+1));
        y_val = spline_polinomios(t_i(k), t(i), t(i+1), y(i), y(i+1), sigma_y(i), sigma_y(i+1));
        x_int = [x_int, x_val];
        y_int = [y_int, y_val];
    end
end
plot(x_int, y_int, 'r-', 'LineWidth', 1.5);
title('Interpolación Spline Cúbica');
xlabel('X'); ylabel('Y');
grid on;
hold off;
% Calcular el área con la fórmula de Gauss
k= length(x);
sum1 = 0;
sum2 = 0;
for i = 1:k-1
    sum1 = sum1 + x(i) * y(i+1);
    sum2 = sum2 + x(i+1) * y(i);
end
Ar = 0.5 * abs(sum1 - sum2)

## Punto 5

In [ ]:
H=[10; 50; 100; 250; 500; 1000; 1500; 2000; 2500; 5000; 10000; 50000; 100000; 500000; 1000000; 1500000];
a=length(H);
P=zeros(a,1);
T=zeros(a,1);
S=zeros(a,1);


for i=1:a
    P(i)=abs(Punto_medio(0,1,H(i))-pi);
    T(i)=abs(Trapecio(0,1,H(i))-pi);
    S(i)=abs(Simpson(0,1,H(i))-pi);
end

disp('Errores absolutos obtenidos al aplicar las reglas de Punto Medio, Trapecio y Simpson');
disp('Tamaño Partición ||     Punto Medio     ||     Trapecio     ||     Simpson')
disp([H, P, T, S]);

figure()
loglog(H,P,'-o',H,T,'-o',H,S,'-o','LineWidth',1.5);
title('Gráficas de errores vs tamaño de la partición','FontSize',12);
legend('Punto Medio','Trapecio', 'Simpson','Location', 'best');
xlabel('log(n)','FontSize', 12);
ylabel('log(error absoluto)','FontSize', 12);
grid("on");

[L, int, err, h]=Romberg(0,1,8);

figure();
loglog(h,err, '-o','LineWidth', 1.5);
title('Gráfica de error vs tamaño del paso usando Método de Romberg','FontSize', 12);
xlabel('log(h)','FontSize', 12);
ylabel('log(error absoluto)','FontSize', 12);
grid('on');

disp([h, err]);
function F=Punto_medio(a,b,N)
    f = @(x) 4/(x.^2 + 1);
    h=(b-a)/N;
    F=0;
    for i=1:N
       F=F+f(a+(i-1)*h+h/2);
    end
    F=h*F;
end

function F=Trapecio(a,b,N)
    f = @(x) 4/(x.^2 + 1);
    h=(b-a)/N;
    F=f(a)+f(b);
    for i=1:N-1
       F=F+2*f(a+i*h);
    end
    F=h*F/2;
end

function F=Simpson(a,b,N)
    f = @(x) 4/(x.^2 + 1);
    h=(b-a)/N;
    F=f(a)+f(b)+4*f(a+h);
    for i=2:N/2
       F=F+2*f(a+(2*i-2)*h)+4*f(a+(2*i-1)*h);
    end
    F=h*F/3;
end

function [R, int, err, h]=Romberg(a,b,N)
    f = @(x) 4/(x.^2 + 1);
    h=zeros(N+1,1);
    dx=b-a;
    err=zeros(N+1,1);
    R=zeros(N+1,N+1);
    R(1,1)=dx*(f(a)+f(b))/2;
    for i=1:N
    h(i)=dx;
    dx=dx/2;
    s=0;
        for k=1:2^(i-1)
            s=s+f(a+(2*k-1)*dx);
        end
    R(i+1,1)=R(i,1)/2 + dx*s;
        for j=1:i
            R(i+1,j+1)=R(i+1,j) + (R(i+1,j)-R(i,j))/(4^j - 1);
        end
    err(i)=abs(R(i,i)-pi);
    end
    h(N+1)=dx;
    int=R(N+1,N+1);
end